In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import tensorflow as tf
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable 

In [2]:
df=pd.read_csv('../input/creditcardfraud/creditcard.csv')
df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [3]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
df_fraud=df['Class']


In [5]:
df_select=df.drop(['Class'],axis=1)

In [6]:
features=np.array(df_select)
label=np.array(df_fraud)


In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from numpy import where
oversample = SMOTE()
X_r, y = oversample.fit_resample(features, label)
# summarize the new class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y == label)[0]

Counter({0: 284315, 1: 284315})


In [8]:
from sklearn.preprocessing import StandardScaler

X_r2 = StandardScaler().fit_transform(X_r)

In [9]:
X_r2.shape

(568630, 30)

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_r2, y, test_size=0.3)
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

In [11]:
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0],1, X_train_tensors.shape[1]))

y_train_tensors_final = torch.reshape(y_train_tensors,   (y_train_tensors.shape[0],1))

X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0],1, X_test_tensors.shape[1])) 
y_test_tensors_final = torch.reshape(y_test_tensors,   (y_test_tensors.shape[0],1))

In [12]:
X_train_tensors_final

tensor([[[-1.7339,  0.2444, -0.1495,  ..., -0.3777,  0.5057, -0.4080]],

        [[-0.3653,  0.5116, -0.0787,  ...,  0.5295,  0.6380, -0.4158]],

        [[ 0.6165,  0.8445, -0.5111,  ..., -0.1028, -0.2338, -0.4151]],

        ...,

        [[-0.3357,  0.2607, -0.8788,  ...,  0.0821, -0.1967, -0.2256]],

        [[ 0.4927,  0.8468, -0.4697,  ..., -0.2072, -0.2881, -0.2907]],

        [[-0.1902, -0.1067, -0.6035,  ...,  0.5028,  0.0612, -0.4284]]])

In [13]:
print("Training Shape", X_train_tensors_final.shape, y_train_tensors_final.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 


Training Shape torch.Size([398041, 1, 30]) torch.Size([398041, 1])
Testing Shape torch.Size([170589, 1, 30]) torch.Size([170589])


In [14]:
device=torch.device('cuda')
def get_lstm_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device)*0.01

    def three():
        return (normal((num_inputs, num_hiddens)),
                normal((num_hiddens, num_hiddens)),
                torch.zeros(num_hiddens, device=device))

    W_xi, W_hi, b_i = three()  
    W_xf, W_hf, b_f = three()  
    W_xo, W_ho, b_o = three()  
    W_xc, W_hc, b_c = three()  

    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)

    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
              b_c, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),
            torch.zeros((batch_size, num_hiddens), device=device))

def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
     W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid((X @ W_xi) + (H @ W_hi) + b_i)
        F = torch.sigmoid((X @ W_xf) + (H @ W_hf) + b_f)
        O = torch.sigmoid((X @ W_xo) + (H @ W_ho) + b_o)
        C_tilda = torch.tanh((X @ W_xc) + (H @ W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * torch.tanh(C)
        Y = (H @ W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, C)



In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes 
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        #self.sigm = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        
        hn = hn.view(-1, self.hidden_size) 
        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out) 
        out = self.fc(out) 
        
        return out


In [16]:

h_0 = Variable(torch.zeros(2, X_train_tensors_final.size(0), 7)) #hidden state
h_0.shape

torch.Size([2, 398041, 7])

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyModel(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(MyModel, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.label = nn.Linear(hidden_size, num_classes)


    def forward(self,x):


        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device))


        output, (final_hidden_state, final_cell_state) = self.lstm(x, (h_0, c_0))
        final_hidden_state = final_hidden_state.view(-1, self.hidden_size)

        return self.label(final_hidden_state) 

In [18]:
X_train_tensors_final.size

<function Tensor.size>

In [19]:
num_epochs = 30
learning_rate = 0.01

input_size = 30
hidden_size = 7
num_layers = 1

num_classes = 1  
model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class

In [20]:
criterion = torch.nn.MSELoss().cuda() if torch.cuda.is_available() else torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [21]:
y_train_tensors_final.shape

torch.Size([398041, 1])

In [22]:
print(X_train_tensors_final.shape)
print(y_train_tensors_final.shape)
X_test_tensors_final
y_test_tensors_final

torch.Size([398041, 1, 30])
torch.Size([398041, 1])


tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])

In [23]:

outputs=model(X_train_tensors_final)
#loss = criterion(outputs, y_train_tensors_final)

In [24]:
outputs.shape

torch.Size([398041, 1])

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
  outputs = model(X_train_tensors_final).to(device)
 #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors_final.to(device))
 
  loss.backward()
 
  optimizer.step() 
  if epoch % 10 == 0:
    #scores = outputs.evaluate(X_test_tensors, Y_test_tensors, verbose=0)
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
    

Epoch: 0, loss: 0.59293
Epoch: 10, loss: 0.14926
Epoch: 20, loss: 0.07860


In [26]:
torch.save(model.state_dict(), "/kaggle/working/w.pth")

In [27]:

# Load the model that we saved at the end of the training loop 
model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1])
path = "w.pth" 
model.load_state_dict(torch.load(path)) 

running_accuracy = 0 
total = 0 

model
    

LSTM1(
  (lstm): LSTM(30, 7, batch_first=True)
  (fc_1): Linear(in_features=7, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

In [28]:
y_test_tensors_final

tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])

In [29]:
m = nn.Softmax(dim=1)
input1 = torch.randn(2, 3)
m(input1)


tensor([[0.1152, 0.0879, 0.7969],
        [0.1255, 0.2218, 0.6527]])

In [30]:
X_test_tensors_final.shape

torch.Size([170589, 1, 30])

In [31]:
model(X_test_tensors_final)

tensor([[0.4948],
        [0.1551],
        [0.7121],
        ...,
        [0.8907],
        [0.4395],
        [0.0827]], grad_fn=<AddmmBackward0>)

In [32]:
y_pred_tag =torch.round(model(X_test_tensors_final))

In [33]:
y_pred_tag

tensor([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]], grad_fn=<RoundBackward0>)

In [34]:
y_test_tensors_final

tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])

In [35]:
y_pred_tag[0]!=y_test_tensors_final[0]

tensor([True])

In [36]:
k=0
for i in range(0,y_pred_tag.shape[0]):
    if y_pred_tag[i]==y_test_tensors_final[i]:
        k=k+1

In [37]:
print('accuracy:',k/y_pred_tag.shape[0])

accuracy: 0.9432378406579557
